In [1]:
# Cell 1: Clone STG-NF và AlphaPose
!git clone https://github.com/orhir/STG-NF.git /kaggle/working/STG-NF
!git clone https://github.com/MVIG-SJTU/AlphaPose.git /kaggle/working/AlphaPose

Cloning into '/kaggle/working/STG-NF'...
remote: Enumerating objects: 449, done.
remote: Counting objects: 100% (449/449), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 449 (delta 284), reused 423 (delta 265), pack-reused 0 (from 0)
Receiving objects: 100% (449/449), 827.77 KiB | 19.25 MiB/s, done.
Resolving deltas: 100% (284/284), done.
Cloning into '/kaggle/working/AlphaPose'...
remote: Enumerating objects: 2749, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 2749 (delta 4), reused 0 (delta 0), pack-reused 2740 (from 2)
Receiving objects: 100% (2749/2749), 118.82 MiB | 36.22 MiB/s, done.
Resolving deltas: 100% (1379/1379), done.


In [2]:
%cd /kaggle/working/AlphaPose
!git clone https://github.com/YuliangXiu/PoseFlow.git

/kaggle/working/AlphaPose
Cloning into 'PoseFlow'...
remote: Enumerating objects: 165, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 165 (delta 33), reused 25 (delta 9), pack-reused 110 (from 1)
Receiving objects: 100% (165/165), 9.52 MiB | 37.21 MiB/s, done.
Resolving deltas: 100% (65/65), done.


In [3]:
%%capture
%cd /kaggle/working/AlphaPose
!pip install "numpy==1.26.4"
!pip install cython pyyaml yacs tqdm matplotlib cython-bbox
!python setup.py build develop

In [4]:
%%capture
%cd /kaggle/working/AlphaPose

!mkdir -p detector/yolo/data
!wget -O detector/yolo/data/yolov3-spp.weights "https://pjreddie.com/media/files/yolov3-spp.weights"

# Tạo thư mục chứa weights (nếu chưa có)
!mkdir -p pretrained_models
# Copy file .pth bạn đã upload vào Kaggle Input
!cp /kaggle/input/pretrained-models/fast_421_res152_256x192.pth \
    pretrained_models/fast_421_res152_256x192.pth

In [5]:
%%capture
%cd /kaggle/working/AlphaPose/PoseFlow
!pip install tqdm opencv-python opencv-contrib-python
!pip install "numpy==1.26.4" "matplotlib<3.8" --force-reinstall

In [6]:
# chuyển xrange → range
!sed -i 's/xrange/range/g' /kaggle/working/AlphaPose/PoseFlow/tracker-general.py

# chuyển iteritems() → items()
!sed -i 's/iteritems()/items()/g' /kaggle/working/AlphaPose/PoseFlow/tracker-general.py

In [7]:
%cd /kaggle/working/AlphaPose/PoseFlow

from pathlib import Path

tracker_file = Path("./tracker-general.py")  # đơn giản hơn

text = tracker_file.read_text()
print("Số lần xuất hiện 'xrange' trước khi sửa:", text.count("xrange"))

text = text.replace("xrange", "range")
tracker_file.write_text(text)

print("Đã sửa xong. 'xrange' còn lại:", tracker_file.read_text().count("xrange"))

/kaggle/working/AlphaPose/PoseFlow
Số lần xuất hiện 'xrange' trước khi sửa: 0
Đã sửa xong. 'xrange' còn lại: 0


# TRACKING

In [8]:
import cv2
from pathlib import Path

# =======================
# CẤU HÌNH
# =======================
VIDEO_DIR = Path("/kaggle/input/cuhk-avenue-dataset/Avenue_Dataset/Avenue Dataset/testing_videos")
OUT_ROOT  = Path("/kaggle/working/frames")
OUT_ROOT.mkdir(parents=True, exist_ok=True)

BATCH_SIZE = 21               # số video mỗi batch
RUN_BATCH = [1]               # <<< CHỈ CHẠY BATCH NÀY (batch bắt đầu từ 1)
# Ví dụ:
# RUN_BATCH = [1]          → chỉ chạy batch 1
# RUN_BATCH = [2, 4]       → chạy 2 batch
# RUN_BATCH = list(range(1,6)) → chạy tất cả batch

# =======================
# LẤY TOÀN BỘ VIDEO
# =======================
videos = sorted([v for v in VIDEO_DIR.iterdir() if v.suffix.lower() in [".mp4", ".avi", ".mov"]])
total_videos = len(videos)

print(f"Found {total_videos} videos.")

# =======================
# CHIA THÀNH BATCH
# =======================
batches = [
    videos[i:i+BATCH_SIZE]
    for i in range(0, total_videos, BATCH_SIZE)
]

print(f"Tổng số batch: {len(batches)} (mỗi batch {BATCH_SIZE} video)")
print(f"Batch được chọn để chạy: {RUN_BATCH}")

# =======================
# XỬ LÝ TỪNG BATCH
# =======================
for batch_idx, batch in enumerate(batches, start=1):

    if batch_idx not in RUN_BATCH:
        continue  # bỏ qua batch này

    print(f"\n==============================")
    print(f"      CHẠY BATCH {batch_idx}/{len(batches)}")
    print(f"      Số video: {len(batch)}")
    print(f"==============================")

    for vid in batch:
        vid_name = vid.stem
        out_dir = OUT_ROOT / vid_name
        out_dir.mkdir(parents=True, exist_ok=True)

        print(f"\n=== Extracting: {vid_name} ===")

        cap = cv2.VideoCapture(str(vid))
        frame_id = 0

        # --- trích frame ---
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame_path = out_dir / f"{frame_id:06d}.jpg"
            cv2.imwrite(str(frame_path), frame)
            frame_id += 1

        cap.release()
        print(f"Extracted {frame_id} frames.")

        # --- đổi tên về <id>.jpg ---
        print("Renaming frames...")
        for img in sorted(out_dir.glob("*.jpg")):
            old = img.stem
            if old.isdigit():
                new = f"{int(old)}.jpg"
                img.rename(img.with_name(new))

        print(f"Done video {vid_name}.")

print("\n=== DONE: tất cả batch yêu cầu đã chạy xong ===")

Found 21 videos.
Tổng số batch: 1 (mỗi batch 21 video)
Batch được chọn để chạy: [1]

      CHẠY BATCH 1/1
      Số video: 21

=== Extracting: 01 ===
Extracted 1439 frames.
Renaming frames...
Done video 01.

=== Extracting: 02 ===
Extracted 1211 frames.
Renaming frames...
Done video 02.

=== Extracting: 03 ===
Extracted 923 frames.
Renaming frames...
Done video 03.

=== Extracting: 04 ===
Extracted 947 frames.
Renaming frames...
Done video 04.

=== Extracting: 05 ===
Extracted 1007 frames.
Renaming frames...
Done video 05.

=== Extracting: 06 ===
Extracted 1283 frames.
Renaming frames...
Done video 06.

=== Extracting: 07 ===
Extracted 605 frames.
Renaming frames...
Done video 07.

=== Extracting: 08 ===
Extracted 36 frames.
Renaming frames...
Done video 08.

=== Extracting: 09 ===
Extracted 1175 frames.
Renaming frames...
Done video 09.

=== Extracting: 10 ===
Extracted 841 frames.
Renaming frames...
Done video 10.

=== Extracting: 11 ===
Extracted 472 frames.
Renaming frames...
Done v

In [9]:
import subprocess
from pathlib import Path
import shutil
import json

# ============================================================
# CẤU HÌNH
# ============================================================

POSEFLOW_DIR = Path("/kaggle/working/AlphaPose/PoseFlow")

JSON_ROOT   = Path("/kaggle/input/cuhk-train-alphapose/alphapose_out_test")
FRAMES_ROOT = Path("/kaggle/working/frames")

POSEFLOW_RAW_ROOT = Path("/kaggle/working/poseflow_raw_json")
POSEFLOW_RAW_ROOT.mkdir(parents=True, exist_ok=True)

FINAL_ROOT = Path("/kaggle/working/poseflow_final_json")
FINAL_ROOT.mkdir(parents=True, exist_ok=True)

TMP_JSON_ROOT = Path("/kaggle/working/poseflow_tmp_json")
TMP_JSON_ROOT.mkdir(parents=True, exist_ok=True)

# Lưu visualize ở đây (KHÔNG xóa)
VIS_ROOT = Path("/kaggle/working/poseflow_vis")
VIS_ROOT.mkdir(parents=True, exist_ok=True)

# Batch
BATCH_SIZE = 21
RUN_BATCH  = [1]  # batch bắt đầu từ 1

# ============================================================
# TIỆN ÍCH
# ============================================================

def ensure_tracker_python3():
    tracker_file = POSEFLOW_DIR / "tracker-general.py"
    txt = tracker_file.read_text()
    if "xrange" in txt:
        tracker_file.write_text(txt.replace("xrange", "range"))

def video_id_from_json(json_path: Path) -> str:
    name = json_path.stem
    if name.endswith("_alphapose_results"):
        return name.replace("_alphapose_results", "")
    return name.replace("_alphapose_results", "")

def imgdir_for_video(video_id: str) -> Path:
    d = FRAMES_ROOT / video_id
    if not d.exists():
        raise FileNotFoundError(f"Không tìm thấy frames folder: {d}")
    return d

# ============================================================
# B1: Chạy PoseFlow + xuất visualize
# ============================================================

def run_poseflow(json_path: Path) -> Path:
    ensure_tracker_python3()

    vid = video_id_from_json(json_path)
    img_dir = imgdir_for_video(vid)

    # Copy JSON sang thư mục writable
    tmp_json = TMP_JSON_ROOT / json_path.name
    shutil.copy2(json_path, tmp_json)

    out_json = POSEFLOW_RAW_ROOT / f"{vid}_poseflow_raw.json"

    # Visualize sẽ được lưu cố định tại đây
    vis_dir = VIS_ROOT / vid
    vis_dir.mkdir(parents=True, exist_ok=True)

    cmd = [
        "python", "tracker-general.py",
        "--imgdir", str(img_dir),
        "--in_json", str(tmp_json),
        "--out_json", str(out_json),
        "--visdir", str(vis_dir),
    ]

    subprocess.run(cmd, cwd=str(POSEFLOW_DIR), check=True)
    print(f"[{vid}] PoseFlow raw JSON saved:", out_json)
    print(f"[{vid}] Visualize saved in:", vis_dir)

    return out_json

# ============================================================
# B2: Convert raw PoseFlow JSON -> track-centric format
# ============================================================

def convert_raw_to_trackcentric(raw_json_path: Path) -> dict:
    with open(raw_json_path, "r") as f:
        data = json.load(f)

    tracks = {}

    for img_name, persons in data.items():
        base = img_name.split(".")[0]
        digits = "".join(ch for ch in base if ch.isdigit())
        if digits == "":
            continue

        frame_idx = int(digits)
        frame_key = f"{frame_idx:04d}"

        for p in persons:
            tid = str(p.get("idx", 0))
            tracks.setdefault(tid, {})
            tracks[tid][frame_key] = {
                "keypoints": p["keypoints"],
                "scores": p.get("scores", 0.0),
            }

    return tracks

# ============================================================
# PIPELINE: 1 file AlphaPose -> tracked_person.json
# ============================================================

def process_one(json_path: Path):
    vid = video_id_from_json(json_path)
    print(f"\n=== VIDEO {vid} ===")

    raw_json = run_poseflow(json_path)
    pred_tracks = convert_raw_to_trackcentric(raw_json)

    out_path = FINAL_ROOT / f"{vid}_alphapose_tracked_person.json"
    with open(out_path, "w") as f:
        json.dump(pred_tracks, f)

    print(f"[{vid}] Tracked saved:", out_path)

# ============================================================
# CHẠY THEO BATCH
# ============================================================

json_files = sorted(JSON_ROOT.glob("*_alphapose_results.json"))
total_files = len(json_files)
print("Total keypoint JSON files found:", total_files)

batches = [json_files[i:i+BATCH_SIZE] for i in range(0, total_files, BATCH_SIZE)]
print("Total batches:", len(batches))
print("RUN_BATCH:", RUN_BATCH)

for batch_idx, batch in enumerate(batches, start=1):
    if batch_idx not in RUN_BATCH:
        continue
    print(f"\n===== RUNNING BATCH {batch_idx}/{len(batches)} (size={len(batch)}) =====")
    for jp in batch:
        process_one(jp)

print("\n=== DONE (selected batches) ===")


Total keypoint JSON files found: 21
Total batches: 1
RUN_BATCH: [1]

===== RUNNING BATCH 1/1 (size=21) =====

=== VIDEO 01 ===
Start loading json file...



100%|██████████| 1439/1439 [00:16<00:00, 88.63it/s]


Start pose tracking...



100%|██████████| 1439/1439 [00:00<00:00, 398468.57it/s]


This video contains 75 people.
Export tracking results to json...



...

Start pose tracking...



100%|██████████| 76/76 [00:00<00:00, 152959.26it/s]
/kaggle/working/AlphaPose/PoseFlow/tracker-general.py:232: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap("hsv", num_persons)
  0%|          | 0/76 [00:00<?, ?it/s]

This video contains 20 people.
Export tracking results to json...

Start visualization...



/kaggle/working/AlphaPose/PoseFlow/tracker-general.py:73: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "r-" (-> color='r'). The keyword argument will take precedence.
  plt.plot(np.clip(pose[pairs[idx],0],0,width),np.clip(pose[pairs[idx],1],0,height),'r-',
100%|██████████| 76/76 [00:33<00:00,  2.26it/s]


[21] PoseFlow raw JSON saved: /kaggle/working/poseflow_raw_json/21_poseflow_raw.json
[21] Visualize saved in: /kaggle/working/poseflow_vis/21
[21] Tracked saved: /kaggle/working/poseflow_final_json/21_alphapose_tracked_person.json

=== DONE (selected batches) ===


In [10]:
import zipfile
from pathlib import Path

# ============================================================
# CẤU HÌNH
# ============================================================

FINAL_JSON_ROOT = Path("/kaggle/working/poseflow_final_json")
VIS_ROOT        = Path("/kaggle/working/poseflow_vis")

OUT_ZIP_JSON = Path("/kaggle/working/poseflow_tracked_json.zip")
OUT_ZIP_VIS  = Path("/kaggle/working/poseflow_visualize.zip")

# ============================================================
# ZIP JSON TRACKED
# ============================================================

def zip_folder(src_dir: Path, zip_path: Path):
    with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zf:
        for f in src_dir.rglob("*"):
            if f.is_file():
                zf.write(f, arcname=f.relative_to(src_dir))
    print(f"Zipped: {zip_path}")

zip_folder(FINAL_JSON_ROOT, OUT_ZIP_JSON)

# ============================================================
# ZIP VISUALIZE
# ============================================================

zip_folder(VIS_ROOT, OUT_ZIP_VIS)

print("=== DONE ZIP ===")


Zipped: /kaggle/working/poseflow_tracked_json.zip
Zipped: /kaggle/working/poseflow_visualize.zip
=== DONE ZIP ===
